In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.conv4 = nn.Conv2d(128, 256, 3)
        self.conv5 = nn.Conv2d(256, 512, 1)
        
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm2d(512)

        self.fc1 = nn.Linear(512, 1024)
        self.fc2 = nn.Linear(1024, 512)
        
        # Separate heads for keypoints and bounding box
        self.fc_keypoints = nn.Linear(512, 8)  # 4 keypoints * 2 (x, y)
        self.fc_bbox = nn.Linear(512, 4)      # Bounding box (x_min, y_min, x_max, y_max)
        
        self.drop1 = nn.Dropout(p=0.25)
    
    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.pool(F.relu(self.bn5(self.conv5(x))))

        x = F.adaptive_avg_pool2d(x, 1)
        x = x.view(x.size(0), -1)
        
        x = F.relu(self.fc1(x))
        x = self.drop1(x)
        x = F.relu(self.fc2(x))
        x = self.drop1(x)
        
        # Separate outputs
        keypoints = self.fc_keypoints(x)
        bbox = self.fc_bbox(x)
        
        return keypoints, bbox

In [4]:
from torchsummary import summary

# Assuming your model is named 'Net'
model = Net()

# Move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Print the summary
summary(model, input_size=(3, 224, 224), device=str(device))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 220, 220]           2,432
       BatchNorm2d-2         [-1, 32, 220, 220]              64
         MaxPool2d-3         [-1, 32, 110, 110]               0
            Conv2d-4         [-1, 64, 108, 108]          18,496
       BatchNorm2d-5         [-1, 64, 108, 108]             128
         MaxPool2d-6           [-1, 64, 54, 54]               0
            Conv2d-7          [-1, 128, 52, 52]          73,856
       BatchNorm2d-8          [-1, 128, 52, 52]             256
         MaxPool2d-9          [-1, 128, 26, 26]               0
           Conv2d-10          [-1, 256, 24, 24]         295,168
      BatchNorm2d-11          [-1, 256, 24, 24]             512
        MaxPool2d-12          [-1, 256, 12, 12]               0
           Conv2d-13          [-1, 512, 12, 12]         131,584
      BatchNorm2d-14          [-1, 512,

In [5]:
model = Net()
total_params = sum(p.numel() for p in model.parameters())
print(f'Total parameters: {total_params}')

# To see detailed parameters per layer
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f'{name}: {param.numel()}')

Total parameters: 1579788
conv1.weight: 2400
conv1.bias: 32
conv2.weight: 18432
conv2.bias: 64
conv3.weight: 73728
conv3.bias: 128
conv4.weight: 294912
conv4.bias: 256
conv5.weight: 131072
conv5.bias: 512
bn1.weight: 32
bn1.bias: 32
bn2.weight: 64
bn2.bias: 64
bn3.weight: 128
bn3.bias: 128
bn4.weight: 256
bn4.bias: 256
bn5.weight: 512
bn5.bias: 512
fc1.weight: 524288
fc1.bias: 1024
fc2.weight: 524288
fc2.bias: 512
fc_keypoints.weight: 4096
fc_keypoints.bias: 8
fc_bbox.weight: 2048
fc_bbox.bias: 4
